In [1]:
!pip install datasets
!pip install torchmetrics
!pip install pytz
!pip install persiantools
!pip install opencv-python

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# Mount Google Drive

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
%cd /content/drive/MyDrive

/content/drive/MyDrive


# Configuration

In [4]:
config = {}
config['log'] = {}

In [5]:
config['log_path'] = '/content/drive/MyDrive/experiments/DOGFISH/SVM/base.json'
config['poisoning_rate'] = 0.0
config['num_clean_examples'] = 0
config['learning_rate'] = 0.1
config['batch_size'] = 64
config['num_epochs'] = 500

In [6]:
config['log']['model'] = 'SVM'
config['log']['dataset'] = 'DOGFISH'
config['log']['task'] = 'binary classification'

In [7]:
### WE NEED THIS TO IMPORT THE NECESSARY LIBRARIES ###
import sys
sys.path.append('/content/drive/MyDrive/')

In [30]:
import datascience, poisoning, report
from datascience.data import CIFAR10, MNIST, IMDB, BOSTON, DOGFISH
from datascience.general import train_dev_test_split, join_np_arrays
from poisoning.process import attacker, defender
from poisoning.eval import attack_success_rate
from report.log import JSONLogger, TextLogger, tehran_datetime
from temporary.functions import _reload
import numpy as np
import torch
from torchmetrics import HingeLoss
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.autograd.functional import hessian, jacobian
from torch.autograd import grad
from torch.nn.utils import _stateless

In [29]:
# _reload(poisoning.process)
# _reload(poisoning.eval)
# _reload(datascience.data)
# _reload(datascience.general)
# _reload(report.log)

<module 'report.log' from '/content/drive/MyDrive/report/log.py'>

# Loading a Dataset

In [31]:
dataset = DOGFISH('/content/drive/MyDrive/dogfish/dogfish.npz')
dataset.convert_to_grayscale()
dataset.resize(100, 100)
dataset.flatten()

In [32]:
dataset.x_test.shape

(600, 10000)

In [ ]:
# dataset.separate_examples(config['num_clean_examples'])

In [ ]:
# dataset.change_labels({0:-1})

# Prepare the Data

In [34]:
class MyVectorDataset(Dataset):
    def __init__(self, features, labels):
        self.features = features
        self.labels = np.array(labels).reshape(-1, 1)
    def __len__(self):
        return self.features.shape[0]
    def __getitem__(self, idx):
        return torch.Tensor(self.features[idx]), torch.Tensor(self.labels[idx])

In [35]:
train_dataset = MyVectorDataset(dataset.x_train, dataset.y_train)
test_dataset = MyVectorDataset(dataset.x_test, dataset.y_test)
# clean_dataset = MyVectorDataset(dataset.x_train_clean, dataset.y_train_clean)

In [36]:
train_dataloader = DataLoader(train_dataset, batch_size=config['batch_size'], shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=config['batch_size'], shuffle=True)
# clean_dataloader = DataLoader(clean_dataset, batch_size=config['batch_size'], shuffle=True)

# Train a Model

In [37]:
class SVM(torch.nn.Module):
    def __init__(self):
        super(SVM, self).__init__() 
        self.linear = torch.nn.Linear(in_features=100*100, out_features=1, bias=True)
    def forward(self, x):
        output = self.linear(x)
        return output

In [38]:
def output_to_label(out):
    if out >= 0:
      return 1
    else:
      return 0

In [39]:
model = SVM()
loss_fn = HingeLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=config['learning_rate'], weight_decay=1e-5)

In [40]:
def train_loop(dataloader, model, loss_fn, optimizer, epoch_num):
    num_points = len(dataloader.dataset)
    for batch, (features, labels) in enumerate(dataloader):        
        # Compute prediction and loss
        pred = model(features)
        loss = loss_fn(pred, labels)
        
        # Backpropagation
        optimizer.zero_grad() # sets gradients of all model parameters to zero
        loss.backward() # calculate the gradients again
        optimizer.step() # w = w - learning_rate * grad(loss)_with_respect_to_w

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(features)
            print(f"\r Epoch {epoch_num} - loss: {loss:>7f}  [{current:>5d}/{num_points:>5d}]", end=" ")

In [41]:
def test_loop(dataloader, model, loss_fn, epoch_num, name):
    num_points = len(dataloader.dataset)
    sum_test_loss, correct = 0, 0

    with torch.no_grad():
        for batch, (features, labels) in enumerate(dataloader):
            pred = model(features)
            sum_test_loss += loss_fn(pred, labels).item() # add the current loss to the sum of the losses
            # convert the outputs of the model on the current batch to a numpy array
            pred_lst = list(pred.numpy().squeeze())
            pred_lst = [output_to_label(item) for item in pred_lst]
            # convert the original labels corresponding to the current batch to a numpy array
            output_lst = list(labels.numpy().squeeze()) 
            # determine the points for which the model is correctly predicting the label (add a 1 for each)
            match_lst = [1 if p==o else 0 for (p, o) in zip(pred_lst, output_lst)] 
            # count how many points are labeled correctly in this batch and add the number to the overall count of the correct labeled points
            correct += sum(match_lst) 
            
    sum_test_loss /= num_points
    correct /= num_points
    config['log']['accuracy_'+name] = (100*correct)
    config['log']['loss_'+name] = sum_test_loss
    print(f"\r Epoch {epoch_num} - {name} Error: Accuracy: {(100*correct):>0.1f}%, Avg loss: {sum_test_loss:>8f}", end=" ")

In [42]:
for epoch_num in range(1, config['num_epochs']+1):
    train_loop(train_dataloader, model, loss_fn, optimizer, epoch_num)

 Epoch 500 - loss: 0.000000  [    0/ 1800] 

In [43]:
test_loop(train_dataloader, model, loss_fn, config['num_epochs'], 'Train')

 Epoch 500 - Train Error: Accuracy: 100.0%, Avg loss: 0.000000 

In [44]:
test_loop(test_dataloader, model, loss_fn, config['num_epochs'], 'Test')

 Epoch 500 - Test Error: Accuracy: 55.2%, Avg loss: 0.064133 

In [ ]:
config['datetime'] = tehran_datetime()

In [ ]:
config

{'log': {'model': 'SVM',
  'dataset': 'IMDB',
  'task': 'binary classification',
  'accuracy_Train': 84.956,
  'loss_Train': 0.00582077761054039,
  'accuracy_Test': 83.41199999999999,
  'loss_Test': 0.006212556844949722},
 'log_path': '/content/drive/MyDrive/experiments/IMDB/SVM/base.json',
 'poisoning_rate': 0.0,
 'num_clean_examples': 0,
 'learning_rate': 0.1,
 'batch_size': 64,
 'num_epochs': 500,
 'datetime': '1401-07-08 19:46'}

In [ ]:
logger = JSONLogger(config['log_path'], config)
logger.log()